In [1]:
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import os


c:\users\shantanu\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#load dataset and use features 
path="D:\Program Scripts\Machine Learning\datasets\Flicker8k_Dataset\Data"
def extract(path):
    model = VGG16()  #FROM https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3
    model.layers.pop()#removes last added layer
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    features = dict()
    files=os.listdir(path)
    for i in range(0,len(files)-7):
        filename = path + '/' + files[i]
        image = load_img(filename, target_size=(224, 224))#Alex net Size
        image = img_to_array(image)
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        image = preprocess_input(image)
        feature = model.predict(image, verbose=0)
        image_id = files[i].split('.')[0]
        features[image_id] = feature
    return features
features = extract(path)
print('Extracted Features: %d' % len(features))
# save to file
dump(features, open('features.pkl', 'wb'))

        


Extracted Features: 8091


In [ ]:
#loading text data
def load_text(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text
filename=path+"/Flickr8k.token.txt"
